# CC2 : OncoPlan - Squelette

**Cours :** IA101 - Intelligence Artificielle (EPF 2026)
**Étudiant(s) :** [VOTRE NOM ICI]

Ce notebook est le squelette à compléter pour le devoir OncoPlan. Il couvre :
1.  **Symbolique** : Ontologie RDF et Planification OR-Tools.
2.  **Probabiliste** : Modélisation Bayésienne avec Pyro.
3.  **Bonus** : Traçabilité via Hashage.

## Installation des Dépendances

In [ ]:
!pip install rdflib ortools pyro-ppl torch pandas matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD
from ortools.sat.python import cp_model
import torch
import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, Predictive
from pyro.optim import Adam
import hashlib
import json

# Configuration
sns.set_style("whitegrid")
pyro.set_rng_seed(101)

---

## Partie 1 : Le Pharmacien Symbolique

### 1.1. Ontologie des Médicaments (RDFLib)

**Objectif :** Créer un graphe RDF contenant les médicaments et leurs propriétés (toxicité, incompatibilités).

In [ ]:
# TODO: Créer le Namespace ONCO
# TODO: Initialiser le Graph
# TODO: Définir les classes et propriétés
# TODO: Peupler la base avec les médicaments (Cisplatine, 5-FU, Docetaxel, Gentamicine)

# VOTRE CODE ICI
pass

In [ ]:
def verifier_prescription(protocole_noms, patient_insuffisance_renale):
    """
    Vérifie si une prescription est valide.
    Args:
        protocole_noms (list): Liste de noms de médicaments (str)
        patient_insuffisance_renale (bool): True si le patient a une insuffisance rénale
    Returns:
        list: Liste de messages d'alerte (str)
    """
    alertes = []
    
    # TODO: Vérifier la néphrotoxicité si le patient est insuffisant rénal
    # TODO: Vérifier les incompatibilités entre paires de médicaments
    
    return alertes

# Test (Ne pas modifier)
print("--- Test 1 : Patient Sain, Protocole OK ---")
print(verifier_prescription(["Cisplatine", "5-FU"], False))

print("\n--- Test 2 : Patient Insuffisant Rénal ---")
print(verifier_prescription(["Cisplatine"], True))

print("\n--- Test 3 : Incompatibilité ---")
print(verifier_prescription(["Cisplatine", "Gentamicine"], False))

### 1.2. Planification des Cures (OR-Tools)

**Objectif :** Planifier 4 cycles de chimiothérapie en respectant les contraintes de temps et de ressources.

In [ ]:
def planifier_chimio(nb_cycles=4, duree_cycle=21, jours_admin=[1, 8], capacite_max=3, occupations_existantes={}):
    model = cp_model.CpModel()
    horizon = nb_cycles * duree_cycle + 10 # Marge
    
    # TODO: Définir la variable de décision (Jour de début du traitement)
    
    # TODO: Ajouter les contraintes
    # 1. Pas de dimanche (supposons J1 = Lundi, donc Dimanche = 7, 14...)
    #    Indice : Utilisez AddModuloEquality pour vérifier si le jour est un multiple de 7
    # 2. Respecter la capacité max (occupations_existantes)
    
    # Résolution
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # TODO: Récupérer la valeur de début et calculer le planning complet
        debut = 0 # À remplacer
        planning = [] # À remplir
        return debut, planning
    else:
        return None, []

# Simulation d'occupations (Jours 10, 11, 12 sont complets)
occupations = {10: 3, 11: 3, 12: 3}
debut, planning = planifier_chimio(occupations_existantes=occupations)
print(f"Début optimal du traitement : Jour {debut}")
print(f"Jours d'administration : {planning}")

---

## Partie 2 : Le Médecin Probabiliste (Pyro)

**Objectif :** Modéliser la réponse du patient (Toxicité) en fonction des doses reçues et des observations (Globules Blancs).

In [ ]:
class OncoModel:
    def __init__(self):
        pass
        
    def model(self, doses, observations=None):
        # TODO: Définir le Prior sur le profil du patient (Categorical)
        # 0: Résistant, 1: Normal, 2: Sensible
        
        # TODO: Définir la boucle temporelle
        # toxicite_cumulee évolue selon dose et sensibilité
        # mu_gb (Globules Blancs) dépend de toxicite_cumulee
        
        # TODO: Définir la Likelihood (Observation)
        pass
            
    def guide(self, doses, observations=None):
        # TODO: Définir le guide variationnel pour 'profil'
        pass

# Instanciation
onco_model = OncoModel()

# Données simulées : Patient reçoit 100mg à T0, 0 à T1... 
# Observation : Chute brutale à T1 (J8)
doses_plan = torch.tensor([100.0, 0.0, 0.0]) # J1, J8, J15
observations_reelles = torch.tensor([7800.0, 2500.0]) # J1 Normal, J8 Neutropénie sévère

# Inférence SVI (À décommenter une fois le modèle implémenté)
# pyro.clear_param_store()
# svi = SVI(onco_model.model, onco_model.guide, Adam({"lr": 0.01}), loss=Trace_ELBO())

# IMPORTANT : Pour l'inférence, assurez-vous d'aligner les doses avec les observations disponibles.
# Si vous passez des doses futures sans observations, Pyro tentera d'inférer ces observations manquantes.
# doses_inf = ... (à définir)

# print("Début de l'inférence...")
# for step in range(1000):
#    loss = svi.step(doses_inf, observations_reelles)
#    if step % 200 == 0:
#        print(f"Step {step} : Loss = {loss}")

# Résultats
# post_probs = pyro.param("profil_probs_post").detach()
# print(f"\nProbabilités a posteriori du profil :")
# print(f"Résistant : {post_probs[0]:.2f}")
# print(f"Normal    : {post_probs[1]:.2f}")
# print(f"Sensible  : {post_probs[2]:.2f}")

### 2.2. Prédiction et Prise de Décision

**Objectif :** Comparer le risque de neutropénie sévère (< 2000) pour deux scénarios futurs.

In [ ]:
# TODO: Utiliser Predictive pour simuler l'avenir
# Scénario 1 : Maintien de la dose (100mg)
# Scénario 2 : Réduction de dose (50mg)

# VOTRE CODE ICI
pass

---

## Partie 3 : Bonus Smart Contract

**Objectif :** Garantir l'intégrité du plan de traitement.

In [ ]:
class OncoContract:
    def __init__(self, medecin_id, patient_id):
        self.medecin_id = medecin_id
        self.patient_id = patient_id
        self.plan_hash = None
        self.signatures = {"medecin": False, "patient": False}
        
    def enregistrer_plan(self, plan_data):
        # TODO: Hasher le plan (SHA-256)
        pass
        
    def signer(self, role, cle_privee_simulee):
        # TODO: Enregistrer la signature
        pass
            
    def verifier_execution(self, plan_propose):
        # TODO: Vérifier que le hash correspond et que tout le monde a signé
        return "Non implémenté"

# Test
contract = OncoContract("DrHouse", "P004")
plan_final = {"J1": "Cisplatine 50mg", "J8": "Repos"}
contract.enregistrer_plan(plan_final)
contract.signer("medecin", "key123")
contract.signer("patient", "key456")

print(contract.verifier_execution(plan_final))
print(contract.verifier_execution({"J1": "Cisplatine 100mg"})) # Tentative de fraude